In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\3\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\3\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetLarge\3\NASNetLarge.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[95.0, 86.0, 47.0, 84.0, 7.0, 47.0, 52.0, 37.0, 16.0, 96.0, 16.0, 45.0, 66.0, 37.0, 37.0, 35.0, 40.0, 32.0, 86.0, 80.0, 37.0, 47.0, 45.0, 10.0, 37.0, 37.0, 86.0, 40.0, 66.0, 86.0, 40.0, 47.0, 86.0, 66.0, 40.0, 30.0, 40.0, 40.0, 96.0, 47.0, 80.0, 7.0, 47.0, 66.0, 45.0, 20.0, 45.0, 16.0, 30.0, 96.0, 45.0, 66.0, 37.0, 52.0, 100.0, 100.0, 84.0, 84.0, 20.0, 80.0, 10.0, 35.0, 95.0, 100.0, 100.0, 13.0, 13.0, 100.0, 80.0, 10.0, 39.0, 86.0, 95.0, 4.0, 80.0, 80.0, 95.0, 95.0, 93.0, 93.0, 84.0, 66.0, 32.0, 96.0, 30.0, 100.0, 84.0, 84.0, 96.0, 37.0, 96.0, 7.0, 30.0, 93.0, 100.0, 52.0, 80.0, 13.0, 52.0, 95.0, 45.0, 37.0, 40.0, 37.0, 39.0, 40.0, 52.0, 45.0, 13.0, 96.0, 66.0, 80.0, 47.0, 39.0, 13.0, 4.0, 52.0, 95.0, 16.0, 95.0, 100.0, 45.0, 35.0, 40.0, 80.0, 10.0, 95.0, 32.0, 66.0, 45.0, 66.0, 45.0, 80.0, 37.0, 86.0, 100.0, 66.0, 20.0, 86.0, 100.0, 30.0, 95.0, 47.0, 32.0, 96.0, 13.0, 47.0, 16.0, 84.0, 35.0, 10.0, 20.0, 100.0, 37.0, 80.0, 47.0, 32.0, 32.0, 47.0, 86.0, 95.0, 100.0, 47.0, 95.0, 40.0, 95

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


In [6]:
#要改
NASNetLarge = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetLarge.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetLarge(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 global_average_pooling2d (G  (None, 4032)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 4033      
                                                                 
Total params: 84,920,851
Trainable params: 84,724,183
Non-trainable params: 196,668
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetLarge_loss_3.txt',history.history['loss'])
#要改
np.savetxt('NASNetLarge_valloss_3.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Sun Apr 23 09:56:42 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 348.2445

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\3\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\3\NASNetLarge.ckpt\assets


591/591 [==============================] - 392s 597ms/step - loss: 348.2445 - val_loss: 11891.0254
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 7.3817

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\3\NASNetLarge.ckpt\assets


...

591/591 [==============================] - 382s 647ms/step - loss: 0.1173 - val_loss: 0.1109
Epoch 26/300
591/591 [==============================] - ETA: 0s - loss: 0.0531

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\3\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\3\NASNetLarge.ckpt\assets


591/591 [==============================] - 391s 662ms/step - loss: 0.0531 - val_loss: 0.0996
Epoch 27/300
591/591 [==============================] - 264s 446ms/step - loss: 0.0774 - val_loss: 0.2778
Epoch 28/300
591/591 [==============================] - 261s 442ms/step - loss: 0.0417 - val_loss: 0.3618
Epoch 29/300
591/591 [==============================] - 261s 442ms/step - loss: 0.0543 - val_loss: 0.3763
Epoch 30/300
591/591 [==============================] - 262s 443ms/step - loss: 0.0500 - val_loss: 0.2498
Epoch 31/300
591/591 [==============================] - 261s 442ms/step - loss: 0.0386 - val_loss: 0.2370
Epoch 32/300
591/591 [==============================] - 260s 440ms/step - loss: 0.0385 - val_loss: 0.3529
Epoch 33/300
591/591 [==============================] - 261s 442ms/step - loss: 0.0349 - val_loss: 0.3957
Epoch 34/300
591/591 [==============================] - 259s 439ms/step - loss: 0.0348 - val_loss: 0.6901
Epoch 35/300
591/591 [==============================] - 262

591/591 [==============================] - 262s 444ms/step - loss: 0.0082 - val_loss: 0.2626
Epoch 180/300
591/591 [==============================] - 264s 447ms/step - loss: 0.0150 - val_loss: 0.3170
Epoch 181/300
591/591 [==============================] - 263s 445ms/step - loss: 0.0418 - val_loss: 0.3225
Epoch 182/300
591/591 [==============================] - 264s 446ms/step - loss: 0.0069 - val_loss: 0.2372
Epoch 183/300
591/591 [==============================] - 265s 448ms/step - loss: 0.0095 - val_loss: 0.2551
Epoch 184/300
591/591 [==============================] - 264s 446ms/step - loss: 0.0066 - val_loss: 0.3488
Epoch 185/300
591/591 [==============================] - 265s 449ms/step - loss: 0.0085 - val_loss: 0.3027
Epoch 186/300
591/591 [==============================] - 263s 446ms/step - loss: 0.0074 - val_loss: 0.3209
Epoch 187/300
591/591 [==============================] - 264s 448ms/step - loss: 0.0067 - val_loss: 0.2958
Epoch 188/300
591/591 [============================